In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
#臺中市政府警察局106年11月份交通事故資料
url_10611 = 'https://quality.data.gov.tw/dq_download_csv.php?nid=72293&md5_url=27099e7b514bec2a041fd8d0194fbd69'

#臺中市政府警察局106年10月份交通事故資料
url_10610 = 'https://quality.data.gov.tw/dq_download_csv.php?nid=59792&md5_url=351eb4516b664646e487f4b7ff5d6671'

In [3]:
# 使用pandas讀取網路連結檔案
df_10611 = pd.read_csv(url_10611)
df_10610 = pd.read_csv(url_10610)

# 使用pandas 存檔
#df_10611.to_csv('交通事故10611.csv')
#df_10610.to_csv('交通事故10610.csv')

## 連接資料庫

In [4]:
# 資料庫連接
# CHARSET=UTF8; 和 cn.setencoding(encoding='utf-8') 都要設定

import pyodbc
cn = pyodbc.connect("DRIVER={MySQL ODBC 8.0 ANSI Driver}; SERVER=localhost; PORT=3306;DATABASE=taichung; UID=root; PASSWORD=123456;OPTION=3;CHARSET=UTF8;") 
cn.setencoding(encoding='utf-8')
cursor = cn.cursor()

In [5]:
cursor.execute("use taichung;")
cursor.execute("show tables;")
cursor.fetchall()

[('pls', ), ('qtable', ), ('tc', )]

In [9]:
####### 小資料試用 ######### 
df1 = df_10611[:2]
df1

,年,月,日,時,分,縣市,區,死,受傷,2-30,...,車輛撞擊部位最初,車輛撞擊部位其他,肇事因素個別,肇事因素主要,肇事逃逸,職業,旅次目的,車種,GPS座標X,GPS座標Y
0,2017,11,1,0,7,臺中市,霧峰區,0.0,0.0,0.0,...,8,,21,21,1,4,8,B03,24.073801,120.663218
1,2017,11,1,0,17,臺中市,潭子區,NaN,NaN,NaN,...,NaN,NaN,13,NaN,1,22,NaN,B03,24.201317,120.703163


In [10]:
# 欄位更名、 NaN 更名(似乎內建式空的 無用)
df1 = df1.rename(columns={'2-30': '2_30', '當事者區分(類別)':'當事者區分_類別'})
df1 = df1.replace('nan', 'NULL')
df1

,年,月,日,時,分,縣市,區,死,受傷,2_30,...,車輛撞擊部位最初,車輛撞擊部位其他,肇事因素個別,肇事因素主要,肇事逃逸,職業,旅次目的,車種,GPS座標X,GPS座標Y
0,2017,11,1,0,7,臺中市,霧峰區,0.0,0.0,0.0,...,8,,21,21,1,4,8,B03,24.073801,120.663218
1,2017,11,1,0,17,臺中市,潭子區,NaN,NaN,NaN,...,NaN,NaN,13,NaN,1,22,NaN,B03,24.201317,120.703163


In [11]:
## 建立資料表 的 query
query = 'create table taichung_201711('

for k in range(1):
    for i, j in zip(df1.columns, df1.dtypes):
        if j.name.find("int")==0:
            query = (query + '\n{} int NULL,').format(i)
        elif j.name.find("float")==0:
            query = (query + '\n{} float NULL,').format(i)
        else:
            query = (query + '\n{} varchar(20) NULL,').format(i)
            
query = query[:-1] + ');'   ##去掉最後的 逗點
print(query)

create table taichung_201711(
年 int NULL,
月 int NULL,
日 int NULL,
時 int NULL,
分 int NULL,
縣市 varchar(20) NULL,
區 varchar(20) NULL,
死 float NULL,
受傷 float NULL,
2_30 float NULL,
天候 float NULL,
光線 float NULL,
道路類別 float NULL,
速限 varchar(20) NULL,
道路型態 float NULL,
事故位置 float NULL,
路面鋪裝 float NULL,
路面狀態 float NULL,
路面缺陷 float NULL,
障礙物 float NULL,
視距 float NULL,
號誌種類 float NULL,
號誌動作 float NULL,
分向設施 float NULL,
快車道或一般車道間 float NULL,
快慢車道間 float NULL,
路面邊線 float NULL,
事故類型及型態 float NULL,
主要肇因 float NULL,
受傷程度 varchar(20) NULL,
主要傷處 varchar(20) NULL,
保護裝備 varchar(20) NULL,
行動電話 varchar(20) NULL,
當事者區分_類別 varchar(20) NULL,
車輛用途 varchar(20) NULL,
當事者行動狀態 varchar(20) NULL,
駕駛資格情形 varchar(20) NULL,
駕駛執照種類 varchar(20) NULL,
飲酒情形 varchar(20) NULL,
車輛撞擊部位最初 varchar(20) NULL,
車輛撞擊部位其他 varchar(20) NULL,
肇事因素個別 varchar(20) NULL,
肇事因素主要 varchar(20) NULL,
肇事逃逸 varchar(20) NULL,
職業 varchar(20) NULL,
旅次目的 varchar(20) NULL,
車種 varchar(20) NULL,
GPS座標X float NULL,
GPS座標Y float NULL);


In [7]:
# 先試個有中文和NaN的資料
#cursor.execute("drop table taichung_201711;")
#cursor.execute("create table taichung_201711(區 varchar(20) NULL, 死 float NULL);")
#cursor.execute("select * from taichung_201711;")
#cursor.execute("show tables;")
cursor.execute("describe taichung_201711;")
cursor.fetchall()
cursor.commit()

In [12]:
df1['死'] = df1.死.replace(np.NaN, 'NULL')

In [14]:
### 建資料表 測試寫入有Nan
#cursor.execute("drop table taichung_201711")
for i in df1.index:
    cursor.execute("insert into taichung_201711 values('{}', {});" \
          .format(df1.loc[i][6], df1.loc[i][7]))

In [27]:
cursor.execute("select * from taichung_201711")
cursor.fetchall()
#cursor.commit()

[('?批陸?\x80', 0.0), ('瞏剖??\x80', None)]

### 想要讓insert into也不用手動打
但是字串要加''  判斷是沒有頭緒


In [20]:
a = 'insert into qtable values('
a = a + '{}, '*df1.shape[1]
a = a[:-2] + ');'
a

'insert into qtable values({}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {});'

In [21]:
b = '.format()'
b

'.format()'